In [1]:
import json
import pandas as pd
import numpy as np
import os
from os import path
import time
import requests
import re
from collections import defaultdict

In [2]:
def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page=0&search=&size={}&sortOrder=desc&sortProperty=apr".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

In [3]:
def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=apr".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

In [4]:
# get uptime
size = get_size(1)
pages = size // 100 + 1
# get the validator info
validator = []
for i in range(pages):
    val = get_validator(i,100)
    validator.extend(val)

In [5]:
column = list(validator[0].keys())
remove = ['bls-public-keys','delegations','last-epoch-in-committee','update-height','remainder','voting_power','signed_blocks','blocks_should_sign','uctDate','index','commision-recent-change','average_stake','average_stake_by_bls','active']
new_column = []
for i in column:
    if i not in remove:
        new_column.append(i)
df = pd.DataFrame(validator, columns = new_column)
percent_change = ['rate','max-rate','max-change-rate', 'apr','uptime_percentage']
for name in percent_change:
    df[name] = df[name].apply(lambda c: "{0:.2f}%".format(float(c)*100))
atto_change = ['min-self-delegation','max-total-delegation', 'self_stake', 'total_stake', 'lifetime_reward_accumulated']
for name in atto_change:
    df[name] = df[name].apply(lambda c: float(c)/1e18)
df.to_csv('../csv/validator_info_mainnet.csv', index = False)
print(f"csv file saved in http://analytics.hmny.io/edit/harmony-log-analysis/projects/staking_dashboard/csv/validator_info_mainnet.csv")

csv file saved in http://analytics.hmny.io/edit/harmony-log-analysis/projects/staking_dashboard/csv/validator_info_mainnet.csv


## email address

In [6]:
pattern = re.compile('(.+?@.+?)')
address = []
for i in df['security-contact']:
    if re.match(pattern, i):
        if "&" in i:
            lst = i.split("&")
            for l in lst:
                address.append(l.replace(" ",""))
        else:
            address.append(i)
with open("../csv/address.txt", "w") as f:
    for i in set(address):
        f.write(i)
        f.write("\n")